In [1]:
import os
import json
import ast
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from elasticsearch import Elasticsearch
from datetime import datetime
from geopy.distance import vincenty, Point

cluster = ['ip-172-31-0-107', 'ip-172-31-0-100', 'ip-172-31-0-105', 'ip-172-31-0-106']

sc = SparkContext(appName="trip")
sqlContext = SQLContext(sc)
ssc = StreamingContext(sc, 3)
sc.setLogLevel("WARN")
es = Elasticsearch(cluster, port=9200)


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=PySparkShell, master=spark://ip-172-31-0-103:7077) created by <module> at /usr/local/lib/python2.7/dist-packages/IPython/utils/py3compat.py:288 

In [75]:
driver = sc.textFile("test.txt")
D = driver.map(ast.literal_eval)

In [365]:
def assign(x):
    ctime = x['ctime']
    location = x['location']
    driver = x['id']
    name = x['name']
    p1 = x['p1']
    p2 = x['p2']
    try:
        tmp = datetime.strptime("{}".format(ctime), '%Y-%m-%d %H:%M:%S.%f')
        ctime = tmp.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
    except:
        pass
    
    def nearby(ctime, location, driver, name, p1=None, p2=None):
        cluster = 'ec2-52-27-127-152.us-west-2.compute.amazonaws.com'
        es = Elasticsearch(cluster, port=9200)

        geo_query = { "from" : 0, "size" : 1,
              "query": {
              "filtered": {
                "query" : {
                 "term" : {"status": "wait"}},
                "filter": {
                "geo_distance": {
                    "distance": '3km',
                    "distance_type": "plane", 
                    "location": location }}
              }}}
        
        res = es.search(index='passenger', doc_type='rolling', body=geo_query )

        if len(res['hits']['hits'])>0: 
            passenger = res['hits']['hits'][0]["_source"]
            doc = json.dumps({"status": "pickup", "driver": driver})
            q = '{{"doc": {}}}'.format(doc)
            res = es.update(index='passenger', doc_type='rolling', id=passenger['id'], body=q, ignore=[400,404])
            doc = {"status": "pickup", "driver": driver, "ctime": ctime, "location": location, \
                   'name': name, 'destination': passenger['location'], 'destinationid': passenger['id']}
            if not p1:
                doc['p1'] = passenger['id']
            elif not p2:
                doc['p2'] = passenger['id']
            else:
                pass # It's full!
            doc = json.dumps(doc)
            q = '{{"doc": {}}}, "doc_as_upsert":True'.format(doc)
            res = es.update(index='driver', doc_type='rolling', id=passenger['id'], \
                            body=q, ignore=[400,404])
            return res
        else:
            res = 'none'
        return res
    res = nearby(ctime, location, driver, name, p1, p2)
    return res

In [403]:
idle = D.filter(lambda x: x['status']=='idle')
if idle.count() > 1: idle.map(assign).count()

In [394]:
def pickup(x):
    if not x: return "Hey!"
    ctime = x['ctime']
    location = x['location']
    driver = x['id']
    name = x['name']
    p1 = x['p1']
    p2 = x['p2']
    try:
        tmp = datetime.strptime("{}".format(ctime), '%Y-%m-%d %H:%M:%S.%f')
        ctime = tmp.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
    except:
        pass
    
    def hopOn(ctime, location, driver, name, p1=None, p2=None):
        passenger = p2 if p2 else p1
        p = es.get(index='passenger', doc_type='rolling', id=passenger, ignore=[404, 400])['_source']
        if (vincenty(Point(location), Point(p['location'])).meters < 300):
            doc = {"status": "ontrip", "ctime": ctime, "location": location,\
                   'destination': passenger['destination'], 'destinationid': passenger['destinationid']}
            doc = json.dumps(doc)
            q = '{{"doc": {}}}'.format(doc)
            res = es.update(index='driver', doc_type='rolling', id=driver, \
                        body=q, ignore=[400,404])

            ## For the sake of demo only, so that the dots are not overlaps on the map
            newLoc = [round(location[0] - 0.0001,4), round(location[1] - 0.0001,4)]
            if p2: newLoc_ = [round(location[0] + 0.0001,4), round(location[1] + 0.0001,4)]

            doc = json.dumps({"status": "ontrip", "ctime": ctime, "location": newLoc})
            if p2:
                doc_ = json.dumps({"status": "ontrip", "ctime": ctime, "location": newLoc_, match: p1})
                doc['match'] = p2
                q_ = '{{"doc": {}}}'.format(doc_)
                res = es.update(index='passenger', doc_type='rolling', id=p2, body=q_, ignore=[400,404])

            q = '{{"doc": {}}}'.format(doc)
            res = es.update(index='passenger', doc_type='rolling', id=p1, body=q, ignore=[400,404])
        else:
            res = False
        return res
    result = hopOn(ctime, location, driver, name, p1, p2)
    return result

In [405]:
pickup = D.filter(lambda x: x['status']=='pickup')
if pickup.count() > 1: idle.map(pickup).count()

In [2]:
from elasticsearch import Elasticsearch
cluster = ['ip-172-31-0-107', 'ip-172-31-0-100', 'ip-172-31-0-105', 'ip-172-31-0-106']
es = Elasticsearch(cluster, port=9200)


In [316]:
geo_query = { "from" : 0, "size" : 1,
          "query": {
          "filtered": {
            "query" : {
             "term" : {"status": "wait"}},
            "filter": {
            "geo_distance": {
                "distance": '3km',
                "distance_type": "plane", 
                "location": [40.8083, -73.924] }}
          }}}

res = es.search(index='passenger', doc_type='rolling', body=geo_query )
passenger = res['hits']['hits'][0]['_source']


In [3]:
res = es.get(index='driver', doc_type='rolling', id=1, ignore=[404, 400])

In [5]:
res['found']

True

In [373]:
x = [40.717, -73.9695]

In [377]:
round(x[0] - 0.0001,4)
round(x[1] - 0.0001,4)


-73.9696